<center>

**Vježba 3 - Čišćenje oil spill skupa podataka**

</center>

U ovoj vježbi želimo vidjeti postupke čišćenja o kojima smo govorili na predavanjima. Nakon čišćenja upotrijebit ćeme neke algoritme strojnog učenja nad pročišćenim skupom.

Prvo skinemo oil spill skup podataka u CSV obliku.

In [ ]:
%%bash
wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/oil-spill.csv

Load + brisanje kolona koje imaju manje od 1% jedinstvenih vrijednosti

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('oil-spill.csv', header = None)
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

del df # Pazimo na memoriju... brišemo sve što nam više ne treba

dfn = X.nunique()
stats = pd.DataFrame()
stats['total'] = dfn
stats['perc'] = (stats['total']/X.shape[0])*100
plt.bar(x = X.columns, height = stats['perc'])
plt.title('Precentage of unique values')
plt.xlabel('Column')
plt.ylabel('Percentage')
plt.show()
plt.close()

print("Sparse ====")
sparse = stats[stats['perc'] < 1.0]
for i,row in sparse.iterrows():
    tot = row['total']
    print(f'Column {i} has {tot} unique values')

sparse_cols = list(sparse.index)
X = X.drop(columns = sparse_cols)
stats = stats.drop(index = sparse_cols)

print(f'New number of columns is {len(stats)}')

Obrišemo kolone s malom varijancom

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from numpy import arange

print(f'X shape {X.shape}')
print(f'y shape {y.shape}')

ths = arange(0.0, 0.55, 0.05)
res = []
for th in ths:
    trans = VarianceThreshold(threshold = th)
    X_selected = trans.fit_transform(X)
    res.append(X_selected.shape[1])

plt.plot(ths, res)
plt.show()

In [ ]:
trans = VarianceThreshold(threshold = 0.15)
X = trans.fit_transform(X) # Vratiti će se numpy array
X = pd.DataFrame(X)

Provjera duplih redaka

In [ ]:
dups = X.duplicated()
duprows = dups.any()
print(f'Ima duplih redaka:{duprows}')
if duprows:
    print(X[dups])

Korelacija značajki. Visoko korelirani se trebaju pregledati.

In [ ]:
import seaborn as sns
import numpy as np

X_corr = X.corr()
mask = np.triu(np.ones_like(X_corr, dtype = bool))
cmap = sns.diverging_palette(220, 10, as_cmap=True)

#make the heatmap plot
plt.figure(figsize=(16,9))
sns.heatmap(X_corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.tight_layout()
plt.show()
plt.close()

Skaliranje za ANN, te razdvajanje na train i test

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

#scaler = MinMaxScaler()
scaler = StandardScaler()

X_scal = scaler.fit_transform(X)
X_scal = pd.DataFrame(X_scal)

X_train, X_test, y_train, y_test = train_test_split(X_scal, y, test_size = 0.2)
print(f'Train shapes {X_train.shape} {y_train.shape}')
print(f'Test shapes {X_test.shape} {y_test.shape}')

Klasifikacija s MLP klasifikatorom.

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=1000, verbose=1)
mlp.fit(X_train, y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

Ispis osnovnih metrika.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

k-struko unakrsno testiranje skupa s dummy klasifikatorom (daje random izlaz), logističkim regresorom i MLP klasifikatorom. Ispisujemo točnost rezultata.

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import matplotlib.pyplot as plt

kfr = RepeatedStratifiedKFold(n_splits = 10, n_repeats = 3, random_state = 1)

reg1 = DummyClassifier()

vc = y.value_counts()
cw = {0: vc[0]/len(y), 1: vc[1]/len(y)}
print(f'Class weights {cw}')
#reg2 = LogisticRegression(solver = 'lbfgs', class_weight = cw)
reg2 = LogisticRegression(solver = 'liblinear')

mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=1000)

score1 = cross_val_score(reg1, X_scal, y, scoring = 'roc_auc', cv = kfr)
print(f'dummy {np.mean(score1)}')
score2 = cross_val_score(reg2, X_scal, y, scoring = 'roc_auc', cv = kfr)
print(f'logistic regression {np.mean(score2)}')
score3 = cross_val_score(mlp, X_scal, y, scoring = 'roc_auc', cv = kfr)
print(f'mlp {np.mean(score3)}')

ROC dijagram.

In [ ]:
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay

def roc_display(X, y, classifier, kfold):
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    fig, ax = plt.subplots(figsize=(6,6))
    for fold,(train,test) in enumerate(kfold.split(X, y)):
        xc = X_scal.iloc[train]
        yc = y.iloc[train]
        classifier.fit(xc, yc)
        viz = RocCurveDisplay.from_estimator(
            classifier,
            X_scal.iloc[test],
            y.iloc[test],
            #name=f"ROC fold {fold}",
            alpha=0.3,
            lw=1,
            ax=ax,
        )
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)

    ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(
        mean_fpr,
        mean_tpr,
        color="b",
        label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
        lw=2,
        alpha=0.8,
    )
    ax.get_legend().remove()

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(
        mean_fpr,
        tprs_lower,
        tprs_upper,
        color="grey",
        alpha=0.2,
        label=r"$\pm$ 1 std. dev.",
    )

    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        xlabel="False Positive Rate",
        ylabel="True Positive Rate",
        title=f"Mean ROC curve with variability\n(Positive label 1)",
    )
    ax.axis("square")
    plt.show()

Crtamo ROC dijagram za razne algoritme.

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(10,), activation='relu', solver='adam', max_iter=1000)

roc_display(X_scal, y, mlp, kfr)

In [ ]:
reg2 = LogisticRegression(solver = 'liblinear')

roc_display(X_scal, y, reg2, kfr)